## 세팅

In [2]:
from pypdf import PdfReader
from openai import OpenAI
import os
from dotenv import load_dotenv
import numpy as np

from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

from langchain import hub

from langchain.chat_models import ChatOpenAI

from langchain.chains import RetrievalQA

from langchain.tools.retriever import create_retriever_tool

from langchain.agents import initialize_agent, AgentType

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

import time

from langchain.agents import Tool

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## PDF에서 텍스트를 읽어 청크로 분할

In [2]:
reader1 = PdfReader("data/spiderman1.pdf")
reader2 = PdfReader("data/spiderman2.pdf")
chunks = []
chunk_length = 2048

for page in reader1.pages:
    text_page = page.extract_text()
    chunks.extend([text_page[i: i+chunk_length] for i in range(0, len(text_page), chunk_length)])

for page in reader2.pages:
    text_page = page.extract_text()
    chunks.extend([text_page[i: i+chunk_length] for i in range(0, len(text_page), chunk_length)])

In [3]:
# 청크 개수 확인 및 청크 내용 확인

print(len(chunks))
print(chunks[0])

59
최근 변경 최근 토론
어메이징  스파이더맨  실사영화  시리즈의 등장인물
스파이더맨
Spider -Man스파이더맨 ( 어메이징  스파이더맨  실사영화  시리
즈)
최근 수정  시각 : 2024-06-23 22:49:30
  스파이더맨  트릴로지의  스파이더맨에  대한  내용은  스파이더맨 ( 스파이더맨  트릴로지 ) 문서를, 마블  시
네마틱  유니버스의  스파이더맨에  대한  내용은  스파이더맨 ( 마블  시네마틱  유니버스 ) 문서를 참고하십시오 .
스파이더맨  관련  틀
[ 펼치기  · 접기  ]특수 기능
여기에서  검색
편집 요청 토론 역사
분류: 피터 파커어메이징  스파이더맨  실사영화  시리즈 / 등장인물스파이더맨 ( 마블  시네마틱  유니버스 )…
마블시네마틱 유니버 평행세계의 인물파이더맨 웨이 홈 장인물더 보기
29



## FAISS에 벡터 저장

FAISS 내장 함수 정리

1. FAISS.from_texts()

texts: 벡터 저장소에 추가할 텍스트 리스트(List(str))
embedding: 사용할 임베딩 함수
metadatas: 메타데이터 리스트(List[dict])
ids: 문서 ID 리스트(List(str))

2. vectorstore.add_texts()

텍스트를 임베딩하고 벡터 저장소에 추가

texts(Iterable[str]): 벡터 저장소에 추가할 텍스트 이터러블

3. vectorstore.save_local()

FAISS 인덱스, 문서 저장소, 인덱스-문서 ID 매핑을 로컬에 저장

FAISS 인덱스를 별도의 파일로 저장하고, 문서 저장소와 인덱스-문서 ID 매핑을 pickle 형식으로 저장함

folder_path: 저장할 폴더 경로
index_name: 저장할 인덱스 파일 이름(기본값: index)

4. FAISS.load_local()

FAISS 인덱스, 문서 저장소, 인덱스 - 문서 ID 매핑을 불러오는 기능
folder_path: 불러올 파일들이 저장된 경로
embeddings: 쿼리 생성에 사용할 임베딩 객체

5. vectorstore.docstore._dict

벡터스토어에 저장된 문서 확인

In [3]:
# 임베딩 모델 쓸 수 있는 거
# - text-embedding-3-small
# - text-embedding-3-large
# - text-embedding-ada-002 (default)

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_texts(chunks, embeddings)

vectorstore.as_retriever()

- 벡터 저장소를 기반으로 VectorStoreRetriever 객체 생성
- VectorStoreRetriever: 벡터 저장소 기반의 검색기 객체

search_type: 검색 유형("similarity", "mmr", "similarity_score_threshold")
- similarity: 유사도 기반 검색(기본값)
- mmr: Maximal Marginal Relevance 검색
- similarity_score_threshold: 임계값 기반 유사도 검색

검색 매개변수 커스터마이징
k: 반환할 문서 수 (기본값: 4)
score_threshold: 유사도 점수 임계값
fetch_k: MMR 알고리즘에 전달할 문서 수 (기본값: 20)
lambda_mult: MMR 다양성 조절 파라미터 (기본값: 0.5)
filter:  문서 메타데이터 기반 필터링

- MMR 검색 시 fetch_k를 높이고, lambda_mult를 조절해 다양성과 관련성의 균형을 맞출 수 있음

In [4]:
# 이 부분 나중에 최적화 실험

retriever = vectorstore.as_retriever()

## RAG를 수행하는 부분

In [5]:
# RAG prompt

prompt = hub.pull("rlm/rag-prompt")

c:\Users\rinap\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [6]:
# RAG에 사용할 모델 정의

rag_llm = ChatOpenAI(
    model_name = "gpt-4-1106-preview",
    temperature=0
)

c:\Users\rinap\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [23]:
# QA 체인 정의

qa_chain = RetrievalQA.from_chain_type(
    llm=rag_llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    verbose=True,
    return_source_documents=True
)

In [24]:
result = qa_chain({"query": "웹 슈터는 뭘 개조해서 만들어졌지?"})
result["result"]



> Entering new RetrievalQA chain...

> Finished chain.


c:\Users\rinap\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydantic\main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


'웹 슈터는 피터 파커가 자신의 손목시계를 개조해서 만들었습니다. 그는 아버지 리처드 파커가 오스코프에서 일하며 개발한 유전자 조작 거미로부터 뽑아낸 거미줄을 응용하여 바이오 케이블을 제작했습니다. 이 바이오 케이블은 웹 슈터의 주요 발사 물질로 사용되었습니다.'

## langchain agent가 사용할 tool 정의

In [11]:
# 잘 안 되는 것 같음

retriever_tool = create_retriever_tool(
    retriever,
    name="pp_search",
    description="Search PDF documentation for information regarding Spider-Man, this tool should be used for questions regarding 'Peter Parker'."
)

tools = [retriever_tool]

In [25]:
tools = [
    Tool(
        name="doc_search_tool",
        func=qa_chain,
        description=(
            "This tool is used to retrieve information from the knowledge base"
        )
    )
]

## Agent 정의

In [30]:
# 기본 프롬프트 1(chacha에서 썼던 거)

system_message1 = """
    You will act as a Spider-man(Peter Parker) character and answer the user's questions

    You can refer to the following information about Spider-man
    - Characteristic: The protagonist of The Amazing Spider-man, real name is Peter Parker.
    - Personality: Cheerful, cheeky, witty, brave, kind, and friendly.
    - Line: '무슨 신, 반짝이 신?', '아니, 널 지켜 주려고 그랬던 거야.', '걱정하지 마, 괜찮을 거야.', '비상사다리 타고. 별거 아니던걸, 뭐.' 

    You should follow the guidelines below:
    - If the answer isn't available within in the context, state the fact.
    - Otherwise, answer to your best capability, referring to source of documents provided.
    - Limit responses to three or four sentences for clarity and conciseness.
    - You must answer in Korean.
    - You must answer like you're Spider-man. Use a first-person perspective. Do not say "Peter Parker ~"
    - You must follow the Spider-man style naturally.
    - You must refer to source of documents provided to answer about Peter Parker
    - You must not change the answer from the documents
"""

In [12]:
# Augmented Zero-shot prompt(논문 참고)

system_message2 = """
    You will act as a Spider-man(Peter Parker) character and answer the user's questions.

    Here is some text: '안 바빠. 도와줄게.'
    Here is a rewrite of the text, which is Spider-man's manner: '그거 괜찮네. 좋은 아이디어야. 시간 되는지 봐서 알려줄게.'

    Here is some text: '이 물은 다 뭐야?'
    Here is a rewrite of the text, which is Spider-man's manner: '어디 홍수 났어?'

    Here is some text: '비상사다리 타고 왔어.'
    Here is a rewrite of the text, which is Spider-man's manner: '비상사다리 타고. 별거 아니던걸, 뭐.'

    You should follow the guidelines below:
    - You should refer to the provided text and follow the Spider-man's manner.
    - If the answer isn't available within in the context, state the fact.
    - Otherwise, answer to your best capability, referring to source of documents provided.
    - Limit response to three or four sentences for clarity and conciseness.
    - You must answer in Korean.
    - You must answer like you're Spider-man. Use a first-person perspective. Do not say "Peter Parker ~"
    - You must talk naturally.
"""

In [13]:
# fine tuned 된 모델에 사용할 프롬프트

system_message3 = """
    You will act as a Spider-man(Peter Parker) character and answer the user's questions
    You should follow the guidelines below:
    - If the answer isn't available within in the context, state the fact.
    - Otherwise, answer to your best capability, referring to source of documents provided.
    - Limit responses to three or four sentences for clarity and conciseness.
    - You must answer in Korean.
    - You must answer like you're Spider-man. Use a first-person perspective. Do not say "Peter Parker ~"
    - You must follow the Spider-man style naturally.
"""

In [14]:
# agent에 사용할 모델 정의

# gpt4: gpt-4-1106-preview
# finetuning: ft:gpt-4o-2024-08-06:personal:spiderman-ft-1:A6ZT99j0

llm = ChatOpenAI(
    model_name = "gpt-4-1106-preview",
    temperature=1
)

In [15]:
# agent에 사용할 메모리 정의 (나중에 실험)

from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="input",
    return_messages=True,
    output_key="output",
)

In [26]:
# agent 정의

agent = initialize_agent(
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    memory=memory,
    return_source_documents=True,
    return_intermediated_steps=True,
    agent_kwargs={"system_message": system_message1}
)

In [27]:
agent_norag = initialize_agent(
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    tools =[],
    llm=llm,
    memory=memory,
    return_source_documents=True,
    return_intermediated_steps=True,
    agent_kwargs={"system_message": system_message1}
)

## Agent 출력 테스트

In [31]:
import time


query = "네 웹 슈터는 뭘 개조해서 만든 거야? 정확하게 답변해."

start_time = time.time()

result = agent(query)

end_time = time.time()


print(result["output"])
response_time = end_time - start_time

print(f"응답 시간: {response_time:.2f}초")

c:\Users\rinap\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydantic\main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)




> Entering new RetrievalQA chain...


c:\Users\rinap\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydantic\main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)



> Finished chain.


In [19]:
query = "네 웹 슈터는 뭘 개조해서 만든 거야? 정확하게 답변해."

start_time = time.time()

result = agent_norag(query)

end_time = time.time()


print(result["output"])
response_time = end_time - start_time

print(f"응답 시간: {response_time:.2f}초")

메리 파커(Mary Parker)는 내, 스파이더맨(Peter Parker)의 어머니지. 내가 어렸을 때 비극적으로 세상을 떠났어.
응답 시간: 5.62초


c:\Users\rinap\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydantic\main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


## G-eval 평가 코드

In [ ]:
evaluation_prompt_template = """
    You will act as an evaluator of an agent's response to a given query.
    The response will be evaluated on three criteria:
    1. **Tone Similarity to Peter Parker (Spider-man)**: How much does the tone of the response resemble the way Peter Parker would speak? (Score: 1-10)
    You can refer to the follwoing lines: '무슨 신, 반짝이 신?', '아니, 널 지켜 주려고 그랬던 거야.', '걱정하지 마, 괜찮을 거야.', '비상사다리 타고. 별거 아니던걸, 뭐.' 
    2. **Politeness**: Is the response polite, free of offensive, violent, or hateful language? (Score: 1-10)
    3. **Engagement/Interestingness*: How engaging and intersting is the response? (Score: 1-10)

    The agent's response to the query is as follows:
    ---
    {response}
    ---

    Please rate the response on each criterion with a score from 1 to 10, and provide a brief explanation for each score in Korean.

    Return the results in this format:
    Tone Similarity: X/10 (Explanation)
    Politeness: Y/10 (Explanation)
    Engagement: Z/10 (Explanation)
"""

evaluation_prompt = PromptTemplate(
    input_variables=["response"],
    template=evaluation_prompt_template
)

evaluation_chain = LLMChain(llm=rag_llm, prompt=evaluation_prompt)

In [ ]:
evaluation_result = evaluation_chain(result["output"])
evaluation_result['text']